In [1]:
# Extract as many features as possible from each word clip and add these cols as X for each word df
# inspired from sunamdha branch code
# check if PCA needed or not and create df's

In [2]:
# change below code to accomodate 3 second clip feature extraction

In [ ]:
# each user 11 words - 10 file\
# each file made into 6 more files
# so each user has 11*10*6*3 = 1980 seconds
# old time - 11*10*3 = 330 seconds

In [92]:
file_path = 'augmented_audio/Abhishek1Archives.wavoutput_compressed.wav'
y, sr = librosa.load(file_path, sr=None)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=25)
mfccs.flatten().shape
# mfccs.shape

(4325,)

### 1) MFCC df creation

In [93]:
import os
import librosa
import pandas as pd
import re

def extract_mfcc(file_path, n_mfcc=25):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfccs.flatten()

# Specify the folder path
folder_path = "augmented_audio/"

# List all files in the folder
files = os.listdir(folder_path)

# Create a dictionary to store data for each word
word_data = {}

# Traverse through each file
for file_name in files:
    if file_name.lower().endswith(".wav"):
        # Parse the file name to extract information
        pattern = r'([A-Za-z]+)\d+([A-Za-z]+)'
        # Use re.match to find the pattern in the file name
        match = re.match(pattern, file_name)
        if match:
            # Extract the name and word from the matched groups
            name = match.group(1)
            word = match.group(2)
        
        # Check if the word is already in the dictionary
        if word not in word_data:
            word_data[word] = {'Name': [], 'MFCC': []}
        
        # Load the original audio and extract MFCC
        input_file_path = os.path.join(folder_path, file_name)
        mfccs = extract_mfcc(input_file_path)
        
        # Add data to the dictionary
        word_data[word]['Name'].append(name)
        word_data[word]['MFCC'].append(mfccs)

# Create DataFrames for each word
word_dfs = {}
for word, data in word_data.items():
    df = pd.DataFrame(data)
    word_dfs[word] = df

# Display DataFrames for each word
for word, df in word_dfs.items():
    print(f"\nWord: {word}")
    print(df)


Word: Archives
         Name                                               MFCC
0    Abhishek  [-478.7596, -485.40863, -517.2088, -519.4477, ...
1    Abhishek  [-476.88766, -491.73743, -509.24796, -505.0290...
2    Abhishek  [-474.8518, -476.6819, -489.24863, -485.178, -...
3    Abhishek  [-301.8254, -278.38187, -283.54584, -279.97052...
4    Abhishek  [-476.34637, -481.575, -511.1833, -512.8605, -...
..        ...                                                ...
205  Sunamdha  [-428.64957, -460.32227, -499.84995, -501.0558...
206  Sunamdha  [-291.29794, -271.4109, -275.13693, -272.7974,...
207  Sunamdha  [-431.0204, -464.70657, -519.9623, -529.9959, ...
208  Sunamdha  [-421.66653, -463.69458, -518.5532, -518.9219,...
209  Sunamdha  [-429.2761, -462.8629, -506.7196, -508.18924, ...

[210 rows x 2 columns]

Word: Chores
         Name                                               MFCC
0    Abhishek  [-504.99988, -503.3169, -528.5078, -533.144, -...
1    Abhishek  [-510.47635, -513.416

### 2) Vector Explore

In [112]:
from scipy.io.wavfile import read
from sklearn import preprocessing
import python_speech_features as mfcc
import numpy as np


def extract_features(audio, rate):
    """
    Extracts audio features using Mel Frequency Cepstral Coefficients (MFCC).

    :param audio: Audio data as a 1D array
    :param rate: Sampling rate of the audio
    :return: Combined feature array of MFCC and delta coefficients
    """

    # Calculate MFCC feature coefficients
    mfcc_feature = mfcc.mfcc(audio, rate, 0.025, 0.01, 20, nfft=1200, appendEnergy=True)

    # Scale the MFCC feature coefficients
    mfcc_feature = preprocessing.scale(mfcc_feature)

    # Print the MFCC feature coefficients
    print(mfcc_feature)

    # Calculate delta coefficients
    delta = calculate_delta(mfcc_feature)

    # Concatenate MFCC and delta coefficients
    combined = np.hstack((mfcc_feature, delta))

    # Return the combined feature array
    return combined



def calculate_delta(array):
    """
    Calculates delta values for each row in the given array.

    :param array: numpy array of shape (rows, cols)
    :return: numpy array of shape (rows, 20) containing the calculated delta values
    """

    # Get the number of rows and columns in the array
    rows, cols = array.shape

    # Create an array to store the delta values
    deltas = np.zeros((rows, 20))

    # Define the value of n
    n = 2

    # Iterate over each row in the array
    for i in range(rows):

        # Create a list to store the indices for calculating delta
        index = []

        # Initialize j to 1
        j = 1

        # Find the indices required for delta calculation
        while j <= n:
            if i - j < 0:
                first = 0
            else:
                first = i - j
            if i + j > rows - 1:
                second = rows - 1
            else:
                second = i + j
            index.append((second, first))
            j += 1

        # Calculate delta using the obtained indices
        deltas[i] = (array[index[0][0]] - array[index[0][1]] + (2 * (array[index[1][0]] - array[index[1][1]]))) / 10

    # Return the calculated delta values
    return deltas


sr, audio = read("augmented_audio/Abhishek1Archives.wavoutput_compressed.wav")
vector = extract_features(audio, sr)

[[-0.35258354 -0.48579106 -0.9273716  ... -0.85279546 -0.50121224
   1.3944195 ]
 [-0.376572   -0.47322831 -0.62364485 ... -0.14806805 -0.08863736
   0.3457843 ]
 [-0.3435563  -0.40214976 -0.16559885 ...  0.40752474  0.84646032
   1.63596079]
 ...
 [ 2.84691675  2.83138524 -1.35066921 ... -0.96517579 -0.58464649
   1.6744359 ]
 [ 2.87868559  2.6982364  -1.78161144 ... -0.75655737 -0.20061998
   2.12726983]
 [ 2.6857284   2.74829096 -1.99428088 ...  0.09847981  0.07856444
   1.8082564 ]]


### 3) Vector and MFCC df creation

In [16]:
import os
import librosa
import pandas as pd
import re
import numpy as np
from scipy.io.wavfile import read
from sklearn import preprocessing
import python_speech_features as mfcc

def extract_mfcc(file_path, n_mfcc=25):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfccs.flatten()

# Function to extract features (MFCC and delta coefficients)
def extract_features(audio, rate):
    mfcc_feature = mfcc.mfcc(audio, rate, 0.025, 0.01, 20, nfft=1200, appendEnergy=True)
    mfcc_feature = preprocessing.scale(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature, delta))
    return combined.flatten()

# Function to calculate delta coefficients
def calculate_delta(array):
    rows, cols = array.shape
    deltas = np.zeros((rows, 20))
    n = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= n:
            if i - j < 0:
                first = 0
            else:
                first = i - j
            if i + j > rows - 1:
                second = rows - 1
            else:
                second = i + j
            index.append((second, first))
            j += 1
        deltas[i] = (array[index[0][0]] - array[index[0][1]] + (2 * (array[index[1][0]] - array[index[1][1]]))) / 10
    return deltas

# Specify the folder path
folder_path = "augmented_audio/"

# List all files in the folder
files = os.listdir(folder_path)

# Create a dictionary to store data for each word
word_data = {}

# Traverse through each file
for file_name in files:
    if file_name.lower().endswith(".wav"):
        # Parse the file name to extract information
        pattern = r'([A-Za-z]+)\d+([A-Za-z]+)'
        # Use re.match to find the pattern in the file name
        match = re.match(pattern, file_name)
        if match:
            # Extract the name and word from the matched groups
            name = match.group(1)
            word = match.group(2)
        
        # Check if the word is already in the dictionary
        if word not in word_data:
            word_data[word] = {'Name': [], 'MFCC': [], 'Vector': []}

        # Load the original audio and extract MFCC
        input_file_path = os.path.join(folder_path, file_name)
        mfccs = extract_mfcc(input_file_path)
        
        # Load the original audio
        sr, audio = read(os.path.join(folder_path, file_name))
        
        # Extract features (MFCC and delta coefficients)
        features = extract_features(audio, sr)
        
        # Add data to the dictionary
        word_data[word]['Name'].append(name)
        word_data[word]['MFCC'].append(mfccs)
        word_data[word]['Vector'].append(features)  # Add the vector values

# Create DataFrames for each word
word_dfs = {}
for word, data in word_data.items():
    df = pd.DataFrame(data)
    word_dfs[word] = df

# Display DataFrames for each word
for word, df in word_dfs.items():
    print(f"\nWord: {word}")
    print(df)


Word: Archives
         Name                                               MFCC  \
0    Abhishek  [-478.7596, -485.40863, -517.2088, -519.4477, ...   
1    Abhishek  [-476.88766, -491.73743, -509.24796, -505.0290...   
2    Abhishek  [-474.8518, -476.6819, -489.24863, -485.178, -...   
3    Abhishek  [-301.8254, -278.38187, -283.54584, -279.97052...   
4    Abhishek  [-476.34637, -481.575, -511.1833, -512.8605, -...   
..        ...                                                ...   
205  Sunamdha  [-428.64957, -460.32227, -499.84995, -501.0558...   
206  Sunamdha  [-291.29794, -271.4109, -275.13693, -272.7974,...   
207  Sunamdha  [-431.0204, -464.70657, -519.9623, -529.9959, ...   
208  Sunamdha  [-421.66653, -463.69458, -518.5532, -518.9219,...   
209  Sunamdha  [-429.2761, -462.8629, -506.7196, -508.18924, ...   

                                                Vector  
0    [0.3961706100387658, -0.34142771019396456, 0.4...  
1    [-0.37629499105190284, -0.47234305931625314, 0..

### 4) RowMean Explore

In [34]:
df['RowMean'] = df['MFCC'].apply(lambda row: np.mean(row))
df

,Name,MFCC,RowMean
0,Abhishek,"[-480.65463, -497.3548, -535.50073, -532.78046...",-17.427166
1,Abhishek,"[-476.8575, -504.7569, -529.0653, -525.89166, ...",-14.392970
2,Abhishek,"[-478.2132, -491.50015, -509.10608, -505.96887...",-13.184780
3,Abhishek,"[-313.2713, -282.91855, -277.0186, -278.47644,...",-9.435394
4,Abhishek,"[-479.9057, -494.56827, -531.24927, -534.4995,...",-16.220234
...,...,...,...
205,Sunamdha,"[-472.059, -478.8089, -490.9839, -485.00278, -...",-12.376882
206,Sunamdha,"[-302.49805, -266.92053, -266.44763, -265.0895...",-8.924610
207,Sunamdha,"[-473.43747, -482.2284, -507.2932, -508.00848,...",-15.689633
208,Sunamdha,"[-473.95013, -489.34067, -506.99823, -501.3318...",-14.476277


In [62]:
df['MFCC'][0].shape

(3600,)

### 5) KNN on RowMean

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming df is your DataFrame with 'RowMean' and 'Name' columns
X = df['RowMean'].values.reshape(-1, 1)  # Reshape to make it a 2D array
y = df['Name']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the KNN classifier (you can adjust the n_neighbors parameter)
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Fit the model on the training data
knn_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.33
Confusion Matrix:
[[ 6  3  2]
 [ 5  6  3]
 [10  5  2]]
Classification Report:
              precision    recall  f1-score   support

    Abhishek       0.29      0.55      0.38        11
        Arun       0.43      0.43      0.43        14
    Sunamdha       0.29      0.12      0.17        17

    accuracy                           0.33        42
   macro avg       0.33      0.36      0.32        42
weighted avg       0.33      0.33      0.31        42



In [104]:
X[0].shape

(3600,)

In [105]:
# Assuming df is your DataFrame with 'MFCC' column
mfcc_lengths = df['MFCC'].apply(len)

# Print information about varying lengths
print("Maximum length of MFCC arrays:", mfcc_lengths.max())
print("Minimum length of MFCC arrays:", mfcc_lengths.min())
print("Average length of MFCC arrays:", mfcc_lengths.mean())
print("Standard deviation of MFCC array lengths:", mfcc_lengths.std())

# Display the distribution of MFCC array lengths
print("\nDistribution of MFCC array lengths:")
print(mfcc_lengths.value_counts())

Maximum length of MFCC arrays: 4325
Minimum length of MFCC arrays: 3600
Average length of MFCC arrays: 4117.857142857143
Standard deviation of MFCC array lengths: 328.3042243175015

Distribution of MFCC array lengths:
MFCC
4325    150
3600     60
Name: count, dtype: int64


### 6) KNN on MFCC or Vector [any one at a time] (with padding)

In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming df is your DataFrame with 'Features' and 'Name' columns
X = df['Vector'].tolist()  # Assuming 'Features' is the column with arrays
y = df['Name']

# Find the maximum length of arrays in 'Features'
max_len = max(len(x) for x in X)

# Calculate the mean of each sequence
mean_values = [np.mean(x) for x in X]

# Pad or truncate arrays to the maximum length using the mean value
X_padded = [
    np.pad(x, (0, max_len - len(x)), mode='constant', constant_values=mean_value) if len(x) < max_len else x[:max_len]
    for x, mean_value in zip(X, mean_values)
]

# Convert the list of arrays to a NumPy array
X_np = np.array(X_padded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_np, y, test_size=0.2, random_state=22)

# Initialize the KNN classifier (you can adjust the n_neighbors parameter)
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Fit the model on the training data
knn_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.98
Confusion Matrix:
[[12  0  0]
 [ 0 15  0]
 [ 1  0 14]]
Classification Report:
              precision    recall  f1-score   support

    Abhishek       0.92      1.00      0.96        12
        Arun       1.00      1.00      1.00        15
    Sunamdha       1.00      0.93      0.97        15

    accuracy                           0.98        42
   macro avg       0.97      0.98      0.98        42
weighted avg       0.98      0.98      0.98        42



### 7) Saving KNN on Vector models

In [26]:
import os
import librosa
import pandas as pd
import re
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def extract_mfcc(file_path, n_mfcc=25):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfccs.flatten()

# Function to extract features (MFCC and delta coefficients)
def extract_features(audio, rate):
    mfcc_feature = mfcc.mfcc(audio, rate, 0.025, 0.01, 20, nfft=1200, appendEnergy=True)
    mfcc_feature = preprocessing.scale(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature, delta))
    return combined.flatten()

# Function to calculate delta coefficients
def calculate_delta(array):
    rows, cols = array.shape
    deltas = np.zeros((rows, 20))
    n = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= n:
            if i - j < 0:
                first = 0
            else:
                first = i - j
            if i + j > rows - 1:
                second = rows - 1
            else:
                second = i + j
            index.append((second, first))
            j += 1
        deltas[i] = (array[index[0][0]] - array[index[0][1]] + (2 * (array[index[1][0]] - array[index[1][1]]))) / 10
    return deltas

# Specify the folder path
folder_path = "augmented_audio/"

# List all files in the folder
files = os.listdir(folder_path)

# Create a dictionary to store data for each word
word_data = {}

# Traverse through each file
for file_name in files:
    if file_name.lower().endswith(".wav"):
        # Parse the file name to extract information
        pattern = r'([A-Za-z]+)\d+([A-Za-z]+)'
        # Use re.match to find the pattern in the file name
        match = re.match(pattern, file_name)
        if match:
            # Extract the name and word from the matched groups
            name = match.group(1)
            word = match.group(2)
        
        # Check if the word is already in the dictionary
        if word not in word_data:
            word_data[word] = {'Name': [], 'MFCC': [], 'Vector': []}

        # Load the original audio and extract MFCC
        input_file_path = os.path.join(folder_path, file_name)
        mfccs = extract_mfcc(input_file_path)
        
        # Load the original audio
        sr, audio = read(os.path.join(folder_path, file_name))
        
        # Extract features (MFCC and delta coefficients)
        features = extract_features(audio, sr)
        
        # Add data to the dictionary
        word_data[word]['Name'].append(name)
        word_data[word]['MFCC'].append(mfccs)
        word_data[word]['Vector'].append(features) 

# Create DataFrames for each word
word_dfs = {}
for word, data in word_data.items():
    df = pd.DataFrame(data)
    
    # Calculate mean of MFCCs and add it to the DataFrame
    df['RowMean'] = df['MFCC'].apply(lambda row: np.mean(row))
    
    # Assuming df is your DataFrame with 'Features' and 'Name' columns
    X = df['Vector'].tolist()  # Assuming 'Features' is the column with arrays
    y = df['Name']

    # Find the maximum length of arrays in 'Features'
    max_len = max(len(x) for x in X)

    # Calculate the mean of each sequence
    mean_values = [np.mean(x) for x in X]

    # Pad or truncate arrays to the maximum length using the mean value
    X_padded = [
        np.pad(x, (0, max_len - len(x)), mode='constant', constant_values=mean_value) if len(x) < max_len else x[:max_len]
        for x, mean_value in zip(X, mean_values)
    ]

    # Convert the list of arrays to a NumPy array
    X_np = np.array(X_padded)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_np, y, test_size=0.2, random_state=22)

    # Initialize the KNN classifier (you can adjust the n_neighbors parameter)
    knn_classifier = KNeighborsClassifier(n_neighbors=3)

    # Fit the model on the training data
    knn_classifier.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = knn_classifier.predict(X_test)
    
    # Print the training accuracy
    train_accuracy = accuracy_score(y_test, knn_classifier.predict(X_test))
    print(f"Training accuracy for word '{word}': {train_accuracy:.2%}")
    
    # Save the trained model for each word
    model_filename_word = f'knn_models/{word}_knn_model.joblib'
    joblib.dump(knn_classifier, model_filename_word)
    print(f"Model for word '{word}' saved to {model_filename_word}")
    
    # Store the DataFrame in the word_dfs dictionary
    word_dfs[word] = df

# Display DataFrames for each word
for word, df in word_dfs.items():
    print(f"\nWord: {word}")
    print(df)

Training accuracy for word 'Archives': 97.62%
Model for word 'Archives' saved to knn_models/Archives_knn_model.joblib
Training accuracy for word 'Chores': 100.00%
Model for word 'Chores' saved to knn_models/Chores_knn_model.joblib
Training accuracy for word 'Environment': 100.00%
Model for word 'Environment' saved to knn_models/Environment_knn_model.joblib
Training accuracy for word 'Hour': 95.24%
Model for word 'Hour' saved to knn_models/Hour_knn_model.joblib
Training accuracy for word 'Iron': 97.62%
Model for word 'Iron' saved to knn_models/Iron_knn_model.joblib
Training accuracy for word 'Pronounciation': 100.00%
Model for word 'Pronounciation' saved to knn_models/Pronounciation_knn_model.joblib
Training accuracy for word 'Reciept': 92.86%
Model for word 'Reciept' saved to knn_models/Reciept_knn_model.joblib
Training accuracy for word 'Suite': 97.62%
Model for word 'Suite' saved to knn_models/Suite_knn_model.joblib
Training accuracy for word 'Tomb': 95.24%
Model for word 'Tomb' save

### 8) KNN on MFCC + Vector

In [23]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming df is your DataFrame with 'MFCC' and 'Vector' columns
X_mfcc = df['MFCC'].tolist()  # Assuming 'MFCC' is the column with MFCC arrays
X_vector = df['Vector'].tolist()  # Assuming 'Vector' is the column with arrays
y = df['Name']

# Find the maximum length of arrays in both 'MFCC' and 'Vector'
max_len_mfcc = max(len(x) for x in X_mfcc)
max_len_vector = max(len(x) for x in X_vector)
max_len = max(max_len_mfcc, max_len_vector)

# Pad or truncate arrays to the maximum length for both 'MFCC' and 'Vector'
X_mfcc_padded = [np.pad(x, (0, max_len - len(x)), mode='constant') if len(x) < max_len else x[:max_len] for x in X_mfcc]
X_vector_padded = [np.pad(x, (0, max_len - len(x)), mode='constant') if len(x) < max_len else x[:max_len] for x in X_vector]

# Concatenate 'MFCC' and 'Vector' arrays along the second axis
X_combined = np.concatenate([X_mfcc_padded, X_vector_padded], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Initialize the KNN classifier (you can adjust the n_neighbors parameter)
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Fit the model on the training data
knn_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.88
Confusion Matrix:
[[10  0  1]
 [ 1 12  1]
 [ 2  0 15]]
Classification Report:
              precision    recall  f1-score   support

    Abhishek       0.77      0.91      0.83        11
        Arun       1.00      0.86      0.92        14
    Sunamdha       0.88      0.88      0.88        17

    accuracy                           0.88        42
   macro avg       0.88      0.88      0.88        42
weighted avg       0.89      0.88      0.88        42



In [115]:
# Assuming df is your DataFrame with 'MFCC' column
mfcc_lengths = df['Vector'].apply(len)

# Print information about varying lengths
print("Maximum length of MFCC arrays:", mfcc_lengths.max())
print("Minimum length of MFCC arrays:", mfcc_lengths.min())
print("Average length of MFCC arrays:", mfcc_lengths.mean())
print("Standard deviation of MFCC array lengths:", mfcc_lengths.std())

# Display the distribution of MFCC array lengths
print("\nDistribution of MFCC array lengths:")
print(mfcc_lengths.value_counts())

Maximum length of MFCC arrays: 199
Minimum length of MFCC arrays: 166
Average length of MFCC arrays: 189.57142857142858
Standard deviation of MFCC array lengths: 14.943502624106964

Distribution of MFCC array lengths:
Vector
199    150
166     60
Name: count, dtype: int64


### 9) Trying LSTM on MFCC

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical

# Assuming df is your DataFrame with 'MFCC' and 'Name' columns
X = df['MFCC'].tolist()  # Assuming 'MFCC' is the column with arrays
y = df['Name']

# Convert labels to numerical format
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

# Find the maximum length of arrays in 'MFCC'
max_len = max(len(x) for x in X)

# Reshape 'MFCC' to have a third dimension for timesteps
X_reshaped = np.array([np.expand_dims(x, axis=1) for x in X])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_encoded, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train_onehot = to_categorical(y_train, num_classes=num_classes)
y_test_onehot = to_categorical(y_test, num_classes=num_classes)

# Build an LSTM model
model = Sequential()
model.add(LSTM(units=128, input_shape=(None, X_reshaped.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=128))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_onehot, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_onehot)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (210,) + inhomogeneous part.

### 10) Saving KNN on MFCC models

In [90]:
import os
import librosa
import pandas as pd
import re
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def extract_mfcc(file_path, n_mfcc=30):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfccs.flatten()

# Specify the folder path
folder_path = "augmented_audio/"

# List all files in the folder
files = os.listdir(folder_path)

# Create a dictionary to store data for each word
word_data = {}

# Traverse through each file
for file_name in files:
    if file_name.lower().endswith(".wav"):
        # Parse the file name to extract information
        pattern = r'([A-Za-z]+)\d+([A-Za-z]+)'
        # Use re.match to find the pattern in the file name
        match = re.match(pattern, file_name)
        if match:
            # Extract the name and word from the matched groups
            name = match.group(1)
            word = match.group(2)
        
        # Check if the word is already in the dictionary
        if word not in word_data:
            word_data[word] = {'Name': [], 'MFCC': []}
        
        # Load the original audio and extract MFCC
        input_file_path = os.path.join(folder_path, file_name)
        mfccs = extract_mfcc(input_file_path)
        
        # Add data to the dictionary
        word_data[word]['Name'].append(name)
        word_data[word]['MFCC'].append(mfccs)

# Create DataFrames for each word
word_dfs = {}
for word, data in word_data.items():
    df = pd.DataFrame(data)
    
    # Calculate mean of MFCCs and add it to the DataFrame
    df['RowMean'] = df['MFCC'].apply(lambda row: np.mean(row))
    
    # Train KNN model for each word
    X_word = df['MFCC'].tolist()
    y_word = df['Name']
    
    max_len_word = max(len(x) for x in X_word)
    X_padded_word = [np.pad(x, (0, max_len_word - len(x)), constant_values=np.mean(x)) if len(x) < max_len_word else x[:max_len_word] for x in X_word]
    X_np_word = np.array(X_padded_word)
    
    X_train_word, X_test_word, y_train_word, y_test_word = train_test_split(X_np_word, y_word, test_size=0.2, random_state=42)
    
    knn_classifier_word = KNeighborsClassifier(n_neighbors=3)
    knn_classifier_word.fit(X_train_word, y_train_word)
    
    # Print the training accuracy
    train_accuracy = accuracy_score(y_train_word, knn_classifier_word.predict(X_train_word))
    print(f"Training accuracy for word '{word}': {train_accuracy:.2%}")
    
    # Save the trained model for each word
    model_filename_word = f'knn_models/{word}_knn_model.joblib'
    joblib.dump(knn_classifier_word, model_filename_word)
    print(f"Model for word '{word}' saved to {model_filename_word}")
    
    # Store the DataFrame in the word_dfs dictionary
    word_dfs[word] = df

# Display DataFrames for each word
for word, df in word_dfs.items():
    print(f"\nWord: {word}")
    print(df)

Training accuracy for word 'Archives': 97.02%
Model for word 'Archives' saved to knn_models/Archives_knn_model.joblib
Training accuracy for word 'Chores': 96.43%
Model for word 'Chores' saved to knn_models/Chores_knn_model.joblib
Training accuracy for word 'Environment': 98.21%
Model for word 'Environment' saved to knn_models/Environment_knn_model.joblib
Training accuracy for word 'Hour': 96.43%
Model for word 'Hour' saved to knn_models/Hour_knn_model.joblib
Training accuracy for word 'Iron': 97.02%
Model for word 'Iron' saved to knn_models/Iron_knn_model.joblib
Training accuracy for word 'Pronounciation': 97.62%
Model for word 'Pronounciation' saved to knn_models/Pronounciation_knn_model.joblib
Training accuracy for word 'Reciept': 96.43%
Model for word 'Reciept' saved to knn_models/Reciept_knn_model.joblib
Training accuracy for word 'Suite': 97.02%
Model for word 'Suite' saved to knn_models/Suite_knn_model.joblib
Training accuracy for word 'Tomb': 97.02%
Model for word 'Tomb' saved t

### 11) Dynamic Predict on Vector KNN models

In [49]:
import random
import sounddevice as sd
import numpy as np
import librosa
import joblib
import os
from python_speech_features import mfcc
from sklearn import preprocessing

# Function to record audio
def record_audio(duration=3, sr=22050):
    print("Please read the word.")
    audio_data = sd.rec(int(duration * sr), samplerate=sr, channels=1, dtype=np.float32)
    sd.wait()
    return audio_data.flatten(), sr

# Function to extract features (MFCC and delta coefficients)
def extract_features(audio, rate):
    mfcc_feature = mfcc(audio, rate, 0.025, 0.01, 20, nfft=1200, appendEnergy=True)
    mfcc_feature = preprocessing.scale(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature, delta))
    return combined.flatten()

# Function to calculate delta coefficients
def calculate_delta(array):
    rows, cols = array.shape
    deltas = np.zeros((rows, 20))
    n = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= n:
            if i - j < 0:
                first = 0
            else:
                first = i - j
            if i + j > rows - 1:
                second = rows - 1
            else:
                second = i + j
            index.append((second, first))
            j += 1
        deltas[i] = (array[index[0][0]] - array[index[0][1]] + (2 * (array[index[1][0]] - array[index[1][1]]))) / 10
    return deltas

# Specify the folder path for KNN models
models_folder = 'knn_models/'

# List all available KNN models
model_files = os.listdir(models_folder)
model_files = [f for f in model_files if f.endswith('_knn_model.joblib')]

# Select 5 random words out of available models
selected_words = random.sample(model_files, 7)

# Initialize a dictionary to store user recordings
user_recordings = {}

# Record audio for each selected word
for word_model in selected_words:
    word = word_model.split('_')[0]
    print(f"\nPlease read the word: {word.capitalize()}")

    # Record audio
    audio_data, sr = record_audio()

    # Extract MFCC features
    features = extract_features(audio_data, sr)

    # Store the user's recording and corresponding MFCC features
    user_recordings[word] = {'Audio': audio_data, 'Vector': features}

# Predict the speaker for each word using the corresponding KNN model
for word, recording_data in user_recordings.items():
    # Load the corresponding KNN model
    model_path = os.path.join(models_folder, f'{word}_knn_model.joblib')
    knn_model = joblib.load(model_path)
    
    # Retrieve the number of features expected by the KNN model
    expected_num_features = knn_model._fit_X.shape[1]
    
    # Ensure the input data has the same number of features as the training data
    num_features_recording = len(recording_data['Vector'])
    
    if num_features_recording != expected_num_features:
        # Calculate the mean value of the features
        mean_value = np.mean(recording_data['Vector'])
        
        # Calculate the difference between expected and actual number of features
        num_features_diff = expected_num_features - num_features_recording
        
        if num_features_diff < 0:
            # Truncate the features if the difference is negative
            recording_data['Vector'] = recording_data['Vector'][:expected_num_features]
        else:
            # Pad the features with the mean value to match the expected dimensions
            recording_data['Vector'] = np.pad(recording_data['Vector'], (0, num_features_diff), constant_values=mean_value)
    
    # Predict the speaker using MFCC features
    user_prediction = knn_model.predict([recording_data['Vector']])
    print(f"\nFor word '{word}', predicted speaker: {user_prediction[0]}")


Please read the word: Suite
Please read the word.

Please read the word: Pronounciation
Please read the word.

Please read the word: Wednesday
Please read the word.

Please read the word: Environment
Please read the word.

Please read the word: Iron
Please read the word.

Please read the word: Hour
Please read the word.

Please read the word: Tomb
Please read the word.

For word 'Suite', predicted speaker: Abhishek

For word 'Pronounciation', predicted speaker: Abhishek

For word 'Wednesday', predicted speaker: Arun

For word 'Environment', predicted speaker: Sunamdha

For word 'Iron', predicted speaker: Sunamdha

For word 'Hour', predicted speaker: Sunamdha

For word 'Tomb', predicted speaker: Arun


### 12) Testing diff models on RowMean Feature

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming df is your DataFrame with 'RowMean' and 'Name' columns
X = df['RowMean'].values.reshape(-1, 1)  # Reshape to make it a 2D array
y = df['Name']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize various classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Support Vector Machine': SVC()
}

# Train and evaluate each classifier
for name, classifier in classifiers.items():
    print(f"\nTraining and evaluating {name}...")
    
    # Fit the model on the training data
    classifier.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = classifier.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)
    
    # Print the evaluation metrics
    print(f"Accuracy: {accuracy:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("Classification Report:")
    print(classification_rep)


Training and evaluating Logistic Regression...
Accuracy: 0.31
Confusion Matrix:
[[ 7  3  1]
 [ 6  4  4]
 [12  3  2]]
Classification Report:
              precision    recall  f1-score   support

    Abhishek       0.28      0.64      0.39        11
        Arun       0.40      0.29      0.33        14
    Sunamdha       0.29      0.12      0.17        17

    accuracy                           0.31        42
   macro avg       0.32      0.35      0.30        42
weighted avg       0.32      0.31      0.28        42


Training and evaluating Decision Tree...
Accuracy: 0.43
Confusion Matrix:
[[4 3 4]
 [4 8 2]
 [5 6 6]]
Classification Report:
              precision    recall  f1-score   support

    Abhishek       0.31      0.36      0.33        11
        Arun       0.47      0.57      0.52        14
    Sunamdha       0.50      0.35      0.41        17

    accuracy                           0.43        42
   macro avg       0.43      0.43      0.42        42
weighted avg       0.44   

### 13) Testing diff models on RowMean Feature

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# Assuming df is your DataFrame with 'Features' and 'Name' columns
X = df['MFCC'].tolist()  # Assuming 'Features' is the column with arrays
y = df['Name']

# Find the maximum length of arrays in 'Features'
max_len = max(len(x) for x in X)

# Pad or truncate arrays to the maximum length
X_padded = [np.pad(x, (0, max_len - len(x))) if len(x) < max_len else x[:max_len] for x in X]

# Convert the list of arrays to a NumPy array
X_np = np.array(X_padded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_np, y, test_size=0.2, random_state=42)

# Initialize various classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Support Vector Machine': SVC()
}

# Train and evaluate each classifier
for name, classifier in classifiers.items():
    print(f"\nTraining and evaluating {name}...")
    
    # Fit the model on the training data
    classifier.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = classifier.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)
    
    # Print the evaluation metrics
    print(f"Accuracy: {accuracy:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("Classification Report:")
    print(classification_rep)


Training and evaluating Logistic Regression...
Accuracy: 1.00
Confusion Matrix:
[[11  0  0]
 [ 0 14  0]
 [ 0  0 17]]
Classification Report:
              precision    recall  f1-score   support

    Abhishek       1.00      1.00      1.00        11
        Arun       1.00      1.00      1.00        14
    Sunamdha       1.00      1.00      1.00        17

    accuracy                           1.00        42
   macro avg       1.00      1.00      1.00        42
weighted avg       1.00      1.00      1.00        42


Training and evaluating Decision Tree...
Accuracy: 0.81
Confusion Matrix:
[[ 8  1  2]
 [ 1 12  1]
 [ 0  3 14]]
Classification Report:
              precision    recall  f1-score   support

    Abhishek       0.89      0.73      0.80        11
        Arun       0.75      0.86      0.80        14
    Sunamdha       0.82      0.82      0.82        17

    accuracy                           0.81        42
   macro avg       0.82      0.80      0.81        42
weighted avg     